In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout, Layer, LSTM, Input

In [2]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [3]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [4]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [5]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [6]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [7]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

In [9]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [18]:
trainX_n_elec2 = trainX_n_elec.iloc[:trainX_elec.shape[0], :]
testX_n_elec2 = testX_n_elec.iloc[:testX_elec.shape[0], :]

In [19]:
#trainx_dhw, trainy_dhw = buildDataSet(trainX_dhw, trainY_dhw, 3)
#testx_dhw, testy_dhw = buildDataSet(testX_dhw, testY_dhw, 3)
trainx_elec, trainy_elec = buildDataSet(trainX_elec, trainY_elec, 3)
testx_elec, testy_elec = buildDataSet(testX_elec, testY_elec, 3)
trainx_n_elec, trainy_n_elec = buildDataSet(trainX_n_elec2, trainY_n_elec, 3)
testx_n_elec, testy_n_elec = buildDataSet(testX_n_elec2, testY_n_elec, 3)

In [20]:
print(trainx_n_elec.shape)
print(trainy_n_elec.shape)
print(trainx_elec.shape)
print(trainy_elec.shape)

(2988, 3, 12)
(2988, 1)
(2988, 3, 12)
(2988, 1)


In [23]:
trainx_n_elec = trainx_n_elec.reshape((trainx_n_elec.shape[0], trainx_n_elec.shape[1], trainx_n_elec.shape[2], 1))
testx_n_elec = testx_n_elec.reshape((testx_n_elec.shape[0], testx_n_elec.shape[1], testx_n_elec.shape[2], 1))
trainx_elec = trainx_elec.reshape((trainx_elec.shape[0], trainx_elec.shape[1], trainx_elec.shape[2], 1))
testx_elec = testx_elec.reshape((testx_elec.shape[0], testx_elec.shape[1], testx_elec.shape[2], 1))

In [24]:
print(trainx_n_elec.shape)
print(trainy_n_elec.shape)
print(trainx_elec.shape)
print(trainy_elec.shape)

(2988, 3, 12, 1)
(2988, 1)
(2988, 3, 12, 1)
(2988, 1)


In [25]:
#Prepare Datasets
BATCH_SIZE = 512
source_dataset = tf.data.Dataset.from_tensor_slices((trainx_n_elec, trainy_n_elec)).batch(BATCH_SIZE*2)
da_dataset = tf.data.Dataset.from_tensor_slices((trainx_n_elec, trainy_n_elec, trainx_elec, trainy_elec)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((testx_elec, testy_elec)).batch(BATCH_SIZE*2) #Test Dataset over Target Domain
test_dataset2 = tf.data.Dataset.from_tensor_slices((trainx_elec, trainy_elec)).batch(BATCH_SIZE*2) #Test Dataset over Target (used for training)

### DANN

In [ ]:
@tf.custom_gradient
def gradient_reverse(x, lamda=1.0):
    y = tf.identity(x)
    
    def grad(dy):
        return lamda * -dy, None
    
    return y, grad

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        
    def call(selx, x, lamda=1.0):
        return gradient_reverse(x, lamda)
    
class DANN(Model):
    def __init__(self):
        super().__init__()
        
        # Feature Extractor
        self.feature_extractor_layer0 = LSTM(64, activation='swish', return_sequences=True),
        self.feature_extractor_layer1 = LSTM(64, activation='swish', return_sequences=False)
        
        # Label Predictor
        self.label_predcitor_layer0 = Dense(64, activation='relu')
        self.label_predcitor_layer1 = Dense(1, activation='relu')
        
        # Domain Predictor
        self.domain_predictor_layer0 = GradientReversalLayer()
        self.domain_predictor_layer1 = Dense(64, activation='relu')
        self.domain_predictor_layer2 = Dense(1, activation=None)
        
    def call(self, x ,train=False, source_train=True, lamda=1.0):
        # Featrue Extractor
        x = self.feature_extractor_layer0(x)
        x = self.feature_extractor_layer1(x)
        
        feature = tf.reshape(x, [-1 * 4 * 4 * 64])
        
        # Label Predictor
        if source_train is True:
            feature_slice = feature
        else:
            feature_slice = tf.slice(feature, [0, 0], [feature.shape[0] // 2, -1])
            
        lp_x = self.label_predcitor_layer0(feature_slice)
        l_logits = self.label_predcitor_layer1(lp_x)